In [4]:

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import sys
sys.path.append('..')
import module.dataloader as dataloader
from module.utils import from_xywh_to_min_max
from tqdm import tqdm
from model_architecture import Network


In [19]:
PROPOSAL_SIZE = (128, 128)
BATCH_SIZE = 1200
BALANCE = 0.5

device='cuda'

normalize_only = transforms.Compose([
    transforms.ToPILImage(),  # Convert NumPy array to PIL Image
    transforms.ToTensor(),    # Convert PIL Image to Tensor [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize the tensor
                       std=[0.229, 0.224, 0.225])
])

dataset_test = dataloader.PotholeDataset(
    '../Potholes/annotated-images/',
    '../Potholes/labeled_proposals/',
    '../Potholes/annotated-images/',
    transform=normalize_only, 
    proposals_per_batch=BATCH_SIZE,
    proposal_size=PROPOSAL_SIZE,
    balance=BALANCE,
    split='train'
)
# dataset_test = dataloader.PotholeDataset('../Potholes/annotated-images/', '../Potholes/labeled_proposals/', '../Potholes/annotated-images/', proposals_per_batch=BATCH_SIZE, proposal_size=PROPOSAL_SIZE, balance=BALANCE, split='test')


# train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=1, shuffle=True, num_workers=4)
# val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=1, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1,num_workers=4, shuffle=False)


In [20]:
model = Network(PROPOSAL_SIZE)
model.load_state_dict(torch.load('../models/model_2.pth'))
model = model.to(device)

/tmp/ipykernel_2601450/1737561638.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../models/model_2.pth'))


In [21]:
import torch
import torchvision
from sklearn.metrics import average_precision_score
from tqdm import tqdm

def validate(model, val_loader, device):
    """
    Validate the model using a validation dataloader and calculate accuracy and average precision (AP).

    Args:
        model (torch.nn.Module): The trained model.
        val_loader (torch.utils.data.DataLoader): DataLoader for validation data.
        device (torch.device): The device to run validation on.

    Returns:
        dict: A dictionary containing validation accuracy and AP.
    """
    model.eval()
    val_correct = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for single_val_dict in tqdm(val_loader, total=len(val_loader)):
            # Load data and move to device
            proposal_image_val = single_val_dict['proposal_images'][0].to(device)
            label_val = single_val_dict['labels'][0].to(device)
            proposals = single_val_dict["proposals"][0].to(device).float()  # Ensure proposals are float

            # Ensure labels have the correct shape
            label_val = label_val.squeeze(-1).float()

            # Get model predictions and apply sigmoid
            output = model(proposal_image_val)
            output = torch.sigmoid(output).squeeze(-1)  # Shape: [N]
            # Perform NMS to filter proposals and scores
            nms_indices = torchvision.ops.nms(proposals, output, 0.5)

            # Filter proposals, scores, and labels using NMS indices
            filtered_output = output[nms_indices]
            filtered_labels = label_val[nms_indices]

            # Collect outputs and labels for AP calculation
            all_outputs.append(filtered_output.cpu().numpy())
            all_labels.append(filtered_labels.cpu().numpy())

            # Calculate accuracy
            predicted = filtered_output > 0.5
            val_correct += (filtered_labels == predicted).sum().cpu().item()

    # Flatten all outputs and labels
    all_outputs = torch.cat([torch.tensor(x) for x in all_outputs]).numpy()
    all_labels = torch.cat([torch.tensor(x) for x in all_labels]).numpy()

    # Calculate Average Precision (AP) using sklearn
    ap_score = average_precision_score(all_labels, all_outputs)

    # Calculate validation accuracy
    val_accuracy = val_correct / len(all_labels)

    # Print metrics
    print(f"Validation Accuracy: {val_accuracy * 100:.1f}%")
    print(f"Average Precision (AP): {ap_score:.4f}")

    return {
        'val_acc': val_accuracy,
        'ap': ap_score
    }



import torch
import torchvision
from sklearn.metrics import average_precision_score
from tqdm import tqdm

def validate_per_image(model, val_loader, device):
    """
    Validate the model per image using a validation dataloader and calculate accuracy and average precision (AP).

    Args:
        model (torch.nn.Module): The trained model.
        val_loader (torch.utils.data.DataLoader): DataLoader for validation data.
        device (torch.device): The device to run validation on.

    Returns:
        dict: A dictionary containing average validation accuracy and AP per image.
    """
    model.eval()
    total_accuracy = 0
    total_ap = 0
    image_count = 0

    with torch.no_grad():
        for single_val_dict in tqdm(val_loader, total=len(val_loader)):
            # Load data and move to device (processing one image at a time)
            proposal_image_val = single_val_dict['proposal_images'][0].to(device)
            label_val = single_val_dict['labels'][0].to(device)
            proposals = single_val_dict["proposals"][0]  # Ensure proposals are float
            proposals = [from_xywh_to_min_max(p) for p in proposals.numpy()]
            proposals = torch.tensor(proposals).to(device).float()
            # Ensure labels have the correct shape
            label_val = label_val.squeeze(-1).float()

            # Get model predictions and apply sigmoid
            output = model(proposal_image_val)
            output = torch.sigmoid(output).squeeze(-1)  # Shape: [N]
            # Perform NMS to filter proposals and scores
            nms_indices = torchvision.ops.nms(proposals, output, 0.5)

            # Filter proposals, scores, and labels using NMS indices
            filtered_output = output[nms_indices]
            filtered_labels = label_val[nms_indices]

            # Calculate AP and accuracy for the single image
            if len(filtered_labels) > 0:
                # AP calculation
                ap_score = average_precision_score(
                    filtered_labels.cpu().numpy(), filtered_output.cpu().numpy()
                )
                total_ap += ap_score

                # Accuracy calculation
                predicted = filtered_output > 0.5
                accuracy = (filtered_labels == predicted).sum().cpu().item() / len(filtered_labels)
                total_accuracy += accuracy

            image_count += 1

    # Average metrics across all images
    avg_accuracy = total_accuracy / image_count if image_count > 0 else 0
    avg_ap = total_ap / image_count if image_count > 0 else 0

    # Print metrics
    print(f"Average Validation Accuracy: {avg_accuracy * 100:.1f}%")
    print(f"Average Precision (AP): {avg_ap:.4f}")

    return {
        'val_acc': avg_accuracy,
        'ap': avg_ap
    }


In [22]:
print(validate_per_image(model, test_loader, device=device))

  0%|          | 0/463 [00:00<?, ?it/s]

  0%|          | 2/463 [00:03<09:56,  1.29s/it]/zhome/81/e/154648/repos/deep-cv-test/test-environment/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
 19%|█▉        | 88/463 [00:38<02:48,  2.23it/s]/zhome/81/e/154648/repos/deep-cv-test/test-environment/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
 27%|██▋       | 127/463 [00:55<02:49,  1.98it/s]/zhome/81/e/154648/repos/deep-cv-test/test-environment/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
 45%|████▌     | 209/463 [01:29<01:29,  2.82it/s]/zhome/81/e/154648/repos/deep-cv-test/test-environment/lib64/python3.9/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: 

Average Validation Accuracy: 86.9%
Average Precision (AP): 0.4078
{'val_acc': 0.8686414483346563, 'ap': 0.4077591232973656}


In [ ]:
import cv2
import matplotlib.pyplot as plt 
def visualize_image(image, boxes,labels, proposals=None):
    # Adjust ground truth boxes according to the scale
    
    # Convert color for display
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    
    # Draw Selective Search proposals in green if provided
    if proposals is not None:
        for (x, y, w, h), label in zip(proposals,labels):
            # Adjust Selective Search boxes according to the scale


            x, y, w, h = int(x), int(y), int(w), int(h)
            if label == 1:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            else:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 1)
                
            # cv2.putText(image, (15, 15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Draw ground truth boxes in blue
    for (xmin, ymin, xmax, ymax) in boxes:
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
    
    plt.imshow(image)
    plt.axis('off')
    plt.show()


def visualize_nms_boxes(model, val_loader, device):
    """
    Validate the model per image using a validation dataloader and calculate accuracy and average precision (AP).

    Args:
        model (torch.nn.Module): The trained model.
        val_loader (torch.utils.data.DataLoader): DataLoader for validation data.
        device (torch.device): The device to run validation on.

    Returns:
        dict: A dictionary containing average validation accuracy and AP per image.
    """
    model.eval()
    total_accuracy = 0
    total_ap = 0
    image_count = 0

    with torch.no_grad():
        iterator = iter(val_loader)
        img = next(iterator)
        # Load data and move to device (processing one image at a time)
        proposal_image_val = img['proposal_images'][0].to(device)
        proposal_image = img["image"][0]
        label_val = img['labels'][0].to(device)
        proposals = img["proposals"][0]  # Ensure proposals are float
        proposals = [from_xywh_to_min_max(p) for p in proposals.numpy()]
        proposals = torch.tensor(proposals).to(device).float()
        # Ensure labels have the correct shape
        label_val = label_val.squeeze(-1).float()

        # Get model predictions and apply sigmoid
        output = model(proposal_image_val)
        output = torch.sigmoid(output).squeeze(-1)  # Shape: [N]
        # Perform NMS to filter proposals and scores
        nms_indices = torchvision.ops.nms(proposals, output, 0.5)

        # Filter proposals, scores, and labels using NMS indices
        filtered_output = output[nms_indices]
        filtered_labels = label_val[nms_indices]
        filtered_proposals = proposals[nms_indices]
        
        visualize_image()
        
        

    # Average metrics across all images
    avg_accuracy = total_accuracy / image_count if image_count > 0 else 0
    avg_ap = total_ap / image_count if image_count > 0 else 0